In [1]:
from fastai.vision.all import *
import torchaudio
import librosa
from nnAudio.features.cqt import CQT1992v2
from scipy.signal import tukey


C:\Users\anaconda\envs\Pytorch_19\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class CFG:
    apex=False
    debug=False
    print_freq=100
    num_workers=4
    model_name='tf_efficientnet_b7_ns'
    scheduler='CosineAnnealingLR' # ['ReduceLROnPlateau', 'CosineAnnealingLR', 'CosineAnnealingWarmRestarts']
    epochs=3
    #factor=0.2 # ReduceLROnPlateau
    #patience=4 # ReduceLROnPlateau
    #eps=1e-6 # ReduceLROnPlateau
    T_max=3 # CosineAnnealingLR
    #T_0=3 # CosineAnnealingWarmRestarts
    lr=1e-4
    min_lr=1e-6
    batch_size=48
    weight_decay=1e-6
    gradient_accumulation_steps=1
    max_grad_norm=1000
    qtransform_params={"hop_length": 1024, "n_bins": 64,'fmax':None}    
    seed=42
    target_size=1
    target_col='target'
    n_fold=5
    trn_fold=[0] # [0, 1, 2, 3, 4]
    train=True
    grad_cam=False


In [3]:
df_train = pd.read_csv(r'C:\Users\Kaggle\Pog_Music_Classification/train.csv')
df_test = pd.read_csv(r'C:\Users\Kaggle\Pog_Music_Classification/test.csv')
submission = pd.read_csv(r'C:\Users\Kaggle\Pog_Music_Classification/sample_submission.csv')

In [4]:
train_path = Path('C:/Users/Kaggle/Pog_Music_Classification/train/')
test_path = Path('C:/Users/Kaggle/Pog_Music_Classification/test/')

In [5]:
print(df_train.shape,df_test.shape)

(19922, 5) (5078, 3)


In [6]:
df_train.head()

,song_id,filename,filepath,genre_id,genre
0,10150,010150.ogg,train/010150.ogg,7,Instrumental
1,7358,007358.ogg,train/007358.ogg,2,Punk
2,20573,020573.ogg,train/020573.ogg,5,Folk
3,11170,011170.ogg,train/011170.ogg,12,Old-Time / Historic
4,16662,016662.ogg,train/016662.ogg,1,Rock


In [7]:
import glob
valid_filenames = []
for i in glob.glob(r'C:/Users/Kaggle/Pog_Music_Classification/resampled_16k/train/*.npy'):
    fnames = (i.split('\\')[1])
    valid_filenames.append(fnames.split('.')[0]+'.ogg')

In [8]:
import glob
valid_filenames_test = []
for i in glob.glob(r'C:/Users/Kaggle/Pog_Music_Classification/resampled_16k/test/*.npy'):
    fnames = (i.split('\\')[1])
    valid_filenames_test.append(fnames.split('.')[0]+'.ogg')

In [9]:
df_train = df_train[df_train['filename'].isin(valid_filenames)].reset_index(drop=True)
df_test_valid = df_test[df_test['filename'].isin(valid_filenames_test)].reset_index(drop=True)

In [10]:
### Some files giving exception in cqt
df_train = df_train[df_train['song_id']!=17400].reset_index(drop=True)

In [11]:
print(df_train.shape,df_test.shape)

(19908, 5) (5078, 3)


### Upsample minority classes

In [12]:
df_train['genre_id'].value_counts()

1     3095
0     3071
2     2582
3     1800
4     1756
5     1214
6     1181
7     1044
8      945
9      814
10     796
11     495
12     408
13     306
14     142
15      94
16      94
17      58
18      13
Name: genre_id, dtype: int64

In [13]:
ExtremeMinority = [18]
SomeMinority = [17]
LowMinority = [14,15,16]

# df_train = pd.concat([df_train,df_train[df_train['genre_id'].isin(ExtremeMinority)].sample(n=37,replace=True,random_state=42)],0).reset_index(drop=True)
# df_train = pd.concat([df_train,df_train[df_train['genre_id'].isin(SomeMinority)].sample(n=42,replace=True,random_state=42)],0).reset_index(drop=True)
# df_train = pd.concat([df_train,df_train[df_train['genre_id'].isin(LowMinority)].sample(n=50,replace=True,random_state=42)],0).reset_index(drop=True)

In [14]:
df_train.tail()#,df_train.shape

,song_id,filename,filepath,genre_id,genre
19903,9337,009337.ogg,train/009337.ogg,0,Electronic
19904,8340,008340.ogg,train/008340.ogg,4,Hip-Hop
19905,16248,016248.ogg,train/016248.ogg,4,Hip-Hop
19906,11875,011875.ogg,train/011875.ogg,2,Punk
19907,6579,006579.ogg,train/006579.ogg,1,Rock


In [15]:
df_train['genre_id'].value_counts()

1     3095
0     3071
2     2582
3     1800
4     1756
5     1214
6     1181
7     1044
8      945
9      814
10     796
11     495
12     408
13     306
14     142
15      94
16      94
17      58
18      13
Name: genre_id, dtype: int64

#### condense bottom 8 classes into single class

In [16]:
# df_train['genre_id'] = df_train['genre_id'].apply(lambda x: 5 if x>=5 else x).values

In [17]:
# df_train['genre_id'].value_counts()

### Create Folds

In [18]:
df_train['fold'] = 0

In [19]:
from sklearn.model_selection import StratifiedKFold
N_folds = 5
kf = StratifiedKFold(5,random_state=42,shuffle=True)
fld = 0
for i,(_,val_id) in enumerate(kf.split(df_train['filename'],df_train['genre_id'])):
    df_train.loc[val_id,'fold'] = int(fld)
    fld += 1

In [20]:
df_train.head()

,song_id,filename,filepath,genre_id,genre,fold
0,10150,010150.ogg,train/010150.ogg,7,Instrumental,4
1,7358,007358.ogg,train/007358.ogg,2,Punk,3
2,20573,020573.ogg,train/020573.ogg,5,Folk,2
3,11170,011170.ogg,train/011170.ogg,12,Old-Time / Historic,3
4,16662,016662.ogg,train/016662.ogg,1,Rock,3


In [21]:
df_test.head()

,song_id,filename,filepath
0,7072,007072.ogg,test/007072.ogg
1,10207,010207.ogg,test/010207.ogg
2,20008,020008.ogg,test/020008.ogg
3,10924,010924.ogg,test/010924.ogg
4,21896,021896.ogg,test/021896.ogg


#### Channel wise max: 2.7517, 2.7950

#### Dataset

In [22]:
import albumentations as A
from albumentations.pytorch import ToTensorV2
from torchvision.transforms import Compose
import torchvision

tfms = Compose(
    [torchvision.transforms.CenterCrop((64,1072))])



In [23]:
import torchaudio

### Make noisy label dataset

In [24]:
OOFOut = pd.DataFrame()
for fld in range(5):
    fold_num = fld
    kernel_type= 'densenet121d_MonoSpec_Hop448_Mels160_FTUNE_MIXUP'
    OOF = np.load(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/oof_fold_{fold_num}_probs.npy',allow_pickle=True)
    ID = np.load(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/oof_fold_{fold_num}_fnames.npy',allow_pickle=True)
    OOF = ([item for sublist in OOF for item in sublist])
    OOF = pd.DataFrame(OOF)
    OOF['ID'] = ID
    OOFOut = pd.concat([OOF,OOFOut],0).reset_index(drop=True)


C:\Users\User\AppData\Local\Temp\ipykernel_4448\634915127.py:10: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  OOFOut = pd.concat([OOF,OOFOut],0).reset_index(drop=True)
C:\Users\User\AppData\Local\Temp\ipykernel_4448\634915127.py:10: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  OOFOut = pd.concat([OOF,OOFOut],0).reset_index(drop=True)
C:\Users\User\AppData\Local\Temp\ipykernel_4448\634915127.py:10: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  OOFOut = pd.concat([OOF,OOFOut],0).reset_index(drop=True)
C:\Users\User\AppData\Local\Temp\ipykernel_4448\634915127.py:10: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  OOFOut = pd.concat([OOF,OOFOut],0).reset_index(drop=True)
C:\Users\Use

In [25]:
f = np.zeros((len(df_test_valid),19))
for fld in range(5):
    for kernels in [
        'densenet121d_MonoSpec_Hop448_Mels160_FTUNE_MIXUP'
                   ]:
        fname = np.load(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernels}/test_fold_{fld}_probs.npy',allow_pickle=True)
        fname = np.concatenate(fname)
         
        f += fname/5
        
f = pd.DataFrame(f)
f['ID'] = df_test_valid['song_id']
f.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,ID
0,0.142654,0.288291,0.050361,0.126073,0.003981,0.055225,0.092180,0.103656,0.031969,0.001747,0.088097,0.009276,0.000293,0.004879,0.000654,0.000094,0.000034,0.000523,0.000015,7072
1,0.002031,0.001676,0.000582,0.002106,0.000348,0.000403,0.000207,0.000446,0.001387,0.001435,0.000139,0.000113,0.986183,0.002283,0.000015,0.000007,0.000201,0.000261,0.000178,10207
2,0.266704,0.036227,0.012025,0.439145,0.003712,0.001595,0.185263,0.015590,0.004062,0.000266,0.028224,0.000462,0.001504,0.003280,0.000080,0.000003,0.001758,0.000096,0.000002,20008
3,0.212893,0.047031,0.029301,0.036472,0.002825,0.003494,0.501256,0.005523,0.011252,0.115956,0.007345,0.001443,0.001595,0.016853,0.002085,0.001547,0.000032,0.002982,0.000114,10924
4,0.042109,0.129475,0.094356,0.056827,0.543139,0.025996,0.005192,0.010515,0.021116,0.035165,0.000618,0.000409,0.000287,0.006498,0.005284,0.004139,0.009570,0.008940,0.000365,21896


In [26]:
consolNoisy = pd.concat([OOFOut,f],0).reset_index(drop=True)
consolNoisy.head()

C:\Users\User\AppData\Local\Temp\ipykernel_4448\4161917736.py:1: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  consolNoisy = pd.concat([OOFOut,f],0).reset_index(drop=True)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,ID
0,0.107273,0.110419,0.067785,0.354302,0.058807,0.053394,0.011387,0.023972,0.069342,0.044853,0.012486,0.004587,0.006164,3.231290e-02,2.554879e-03,0.002291,2.774757e-02,0.009656,0.000664,10150
1,0.001319,0.003918,0.002543,0.001308,0.000077,0.000241,0.000272,0.000531,0.001216,0.952048,0.000033,0.000561,0.000037,2.977470e-02,1.896874e-03,0.001439,3.833476e-05,0.002692,0.000056,11198
2,0.386807,0.092747,0.033228,0.008084,0.041854,0.012293,0.059948,0.005167,0.309701,0.014301,0.009484,0.000037,0.000011,1.525782e-03,4.073117e-03,0.019252,2.619769e-07,0.001153,0.000335,13166
3,0.025068,0.001094,0.001839,0.000609,0.960854,0.000214,0.003158,0.000032,0.003609,0.002018,0.001037,0.000001,0.000006,1.910009e-07,8.585370e-07,0.000326,1.609574e-06,0.000017,0.000116,12275
4,0.085827,0.047051,0.023459,0.675443,0.004005,0.009120,0.012726,0.059727,0.007839,0.003343,0.037378,0.006672,0.006419,2.124800e-03,5.385189e-05,0.000007,1.848144e-02,0.000306,0.000018,24102


In [27]:
class POGFiles(torch.utils.data.Dataset):
    
    def __init__(self, df,
                 noisyDF = consolNoisy,
                 train = True,
                 valid = False,
                 transform=tfms,

                ):
        
        self.df = df
        self.train = train
        self.valid = valid
        self.transform = transform
        self.noisyDF = noisyDF
        
    def __len__(self):
        return len(self.df)
    
    def load_file(self, filename):
        if self.train:
            spec = np.load(f'C:/Users/Kaggle/Pog_Music_Classification/spec/448_64/train/{filename}')
        elif self.valid:
            spec = np.load(f'C:/Users/Kaggle/Pog_Music_Classification/spec/448_64/train/{filename}')
        else:
            spec = np.load(f'C:/Users/Kaggle/Pog_Music_Classification/spec/448_64/test/{filename}')
            
        return spec


    def __getitem__(self, index):
        row  = self.df.iloc[index]
        
        samples = self.load_file(row.filename.split('.')[0]+'.npy')
        samples = torch.from_numpy(samples).float()
        
        if self.transform is not None:
            samples = self.transform(samples)
            
        if self.train:
            labels2 = self.noisyDF[self.noisyDF.ID==row.song_id].drop('ID',1).values[0]
            label = torch.tensor(row.genre_id,dtype=torch.long)
            label2 = torch.tensor(labels2).float()
            return samples, (label,label2)
        if self.valid:
            label = torch.tensor(row.genre_id,dtype=torch.long)
            return samples, label
        else:
            return samples

#### Loss

In [28]:
class CombinedLoss:
    "Dice and Focal combined"
    def __init__(self, axis=1, smooth=1., alpha=0.5):
        store_attr()
        self.focal_loss = CrossEntropyLossFlat()
        self.dice_loss =  BCEWithLogitsLossFlat()
        
    def __call__(self, pred, targ):
        if type(targ) is list:
            loss = self.focal_loss(pred, targ[0])*(1-self.alpha) + self.alpha * self.dice_loss(pred, targ[1])
        else:
            loss = self.focal_loss(pred, targ)
        return loss
    
    def decodes(self, x):    return x.argmax(dim=self.axis)
    def activation(self, x): return F.softmax(x, dim=self.axis)

#### FastAI model

In [29]:
import timm
# timm.create_model('resnext50d_32x4d', pretrained=False, in_chans=1)

In [30]:
NCLASS = df_train['genre_id'].nunique()
print(NCLASS)

19


In [31]:
def run():
    oof = df_train.copy()
    oversampleTrain = False
    
    for fold_num in [0,1,2,3,4]:
        print('*****************************************')
        print(f'Training Fold {fold_num}')
        print('*****************************************')

    #with IPyExperimentsPytorch() as exp:
        kernel_type = 'densenet121d_MonoSpec_Hop448_Mels64_Noisy'
        OUTPUT_DIR = f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/'
        if not os.path.exists(OUTPUT_DIR):
            os.makedirs(OUTPUT_DIR)

            
        crit =  CombinedLoss()#LabelSmoothingCrossEntropy(0.02) #CrossEntropyLossFlat() #FocalLoss() #CrossEntropyLossFlat() #FocalLoss() #LabelSmoothingCrossEntropy(0.01)
        batch_size = 48
        n_epochs = 15

        training_fold = df_train.query(f'fold!={fold_num}').reset_index(drop=True, inplace=False)
        
        if oversampleTrain:
            print('--------- Oversampling training dataset -----------')
            training_fold = pd.concat([training_fold,training_fold[training_fold['genre_id'].isin(ExtremeMinority)].sample(n=150,replace=True,random_state=42)],0).reset_index(drop=True)
            training_fold = pd.concat([training_fold,training_fold[training_fold['genre_id'].isin(SomeMinority)].sample(n=150,replace=True,random_state=42)],0).reset_index(drop=True)
            training_fold = pd.concat([training_fold,training_fold[training_fold['genre_id'].isin(LowMinority)].sample(n=100,replace=True,random_state=42)],0).reset_index(drop=True)
                
        train_ds = POGFiles(training_fold,
                            transform=tfms
                           )

        validation_fold = df_train.query(f'fold=={fold_num}').reset_index(drop=True, inplace=False)
        valid_ds = POGFiles(validation_fold,
                            transform=tfms,train=False,valid=True)

        print(f'- Training samples: {len(train_ds)}\n- Validation Samples : {len(valid_ds)}')

        bs = batch_size
        train_dl = torch.utils.data.DataLoader(train_ds, batch_size=bs, num_workers=0,pin_memory=False)
        valid_dl = torch.utils.data.DataLoader(valid_ds, batch_size=bs*2, num_workers=0,shuffle=False,pin_memory=False)

        dls = DataLoaders(train_dl, valid_dl)
        
        
        model = timm.create_model('densenet121d', pretrained=False, in_chans=1)
        model.features.conv0.stride = (1,1)
        model.classifier = nn.Linear(1024,NCLASS)
    
        f1_score = F1Score(average="micro")

        learn = Learner(dls, model,opt_func=Adam,loss_func=crit,metrics=[f1_score],cbs=[SaveModelCallback('f1_score', every_epoch=True,),
                                                                                          CSVLogger(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/{fold_num}logs.csv')]).to_fp16()
        
        learn.fit_one_cycle(n_epochs, 1e-2,wd=1e-3)
        learn.save(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/fold_{fold_num}')
        learn = learn.load(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/fold_{fold_num}')
        
        learn.model.eval()
        test_df = df_test
        test_ds = POGFiles(df_test_valid,
                           transform=tfms,
                           train=False,
                           valid=False
                           )

        test_ds.input_path = Path(test_path)

        bs = batch_size
        test_dl = torch.utils.data.DataLoader(test_ds, batch_size=bs*2, num_workers=0, shuffle=False)
        
        preds = []
        probs = []
        for xb  in progress_bar(test_dl):
            with torch.no_grad():output = learn.model(xb.cuda())
            probs.append(torch.softmax(output.float(),1).squeeze().cpu())
            preds.append(torch.argmax(output.float(),1).squeeze().cpu())
        
        oof_ = []
        for xb,_  in progress_bar(valid_dl):
            with torch.no_grad():output = learn.model(xb.cuda())
            oof_.append(torch.softmax(output.float(),1).squeeze().cpu())
        
        oof_ = [t.detach().numpy() for t in oof_]
        np.save(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/test_fold_{fold_num}_probs.npy',np.array(probs))
        np.save(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/oof_fold_{fold_num}_probs.npy',oof_)
        np.save(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/oof_fold_{fold_num}_fnames.npy',np.array(validation_fold.song_id))        

        preds = torch.cat(preds)
        sample_df = df_test_valid.copy()
        sample_df['target'] = preds
        sample_df.to_csv(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/fold_{fold_num}.csv', index=False)

In [32]:
import warnings
warnings.filterwarnings('ignore')

In [33]:
if __name__=='__main__':
    run()

*****************************************
Training Fold 0
*****************************************
- Training samples: 15926
- Validation Samples : 3982


epoch,train_loss,valid_loss,f1_score,time
0,0.994737,1.835175,0.397539,09:20
1,0.955877,2.081107,0.376444,06:13
2,0.891222,2.288005,0.319186,07:20
3,0.837471,2.952797,0.209694,07:50
4,0.796951,2.032130,0.393019,07:54
5,0.760709,1.786648,0.447263,07:44
6,0.728298,1.553893,0.490206,07:11
7,0.693756,1.562316,0.483928,06:06
8,0.658821,1.654836,0.466600,06:03
9,0.628834,1.489156,0.515821,06:03


*****************************************
Training Fold 1
*****************************************
- Training samples: 15926
- Validation Samples : 3982


epoch,train_loss,valid_loss,f1_score,time
0,0.985520,2.187375,0.322702,06:07
1,0.955567,2.207144,0.288800,06:07
2,0.892492,1.758923,0.420643,06:31
3,0.841473,2.159289,0.361125,06:10
4,0.798732,1.971808,0.374184,06:03
5,0.763638,1.799407,0.426921,06:01
6,0.733419,1.600477,0.478905,06:01
7,0.703534,1.576897,0.496986,06:01
8,0.672924,1.689603,0.489955,06:04
9,0.640252,1.534610,0.507032,06:02


*****************************************
Training Fold 2
*****************************************
- Training samples: 15926
- Validation Samples : 3982


epoch,train_loss,valid_loss,f1_score,time
0,1.006603,3.284524,0.246861,06:02
1,0.968766,5.801810,0.093672,06:01
2,0.905972,2.788170,0.239578,06:01
3,0.851725,2.603739,0.257659,06:02
4,0.812606,2.129405,0.343546,06:02
5,0.774492,1.562134,0.482170,06:01
6,0.741601,4.741330,0.134355,06:02
7,0.710703,2.775758,0.264440,06:01
8,0.681518,1.536647,0.482672,06:02
9,0.650485,1.462021,0.503265,06:02


*****************************************
Training Fold 3
*****************************************
- Training samples: 15927
- Validation Samples : 3981


epoch,train_loss,valid_loss,f1_score,time
0,0.987477,2.988483,0.216277,06:08
1,0.962126,2.253894,0.267270,06:03
2,0.893983,2.102390,0.298417,06:02
3,0.838328,2.112203,0.320774,06:02
4,0.794336,2.161497,0.301181,06:02
5,0.766016,1.509371,0.493595,06:02
6,0.732687,1.460624,0.509169,06:01
7,0.699158,1.472661,0.508415,06:01
8,0.668762,1.606297,0.468726,06:01
9,0.633287,1.603163,0.457172,06:09


*****************************************
Training Fold 4
*****************************************
- Training samples: 15927
- Validation Samples : 3981


epoch,train_loss,valid_loss,f1_score,time
0,1.011163,2.191853,0.302939,06:02
1,0.956247,2.229674,0.314243,06:01
2,0.891938,2.278697,0.350414,06:01
3,0.839503,1.860787,0.407687,06:01
4,0.797983,1.659535,0.448129,06:01
5,0.765218,1.762158,0.418488,06:01
6,0.732543,1.961740,0.364230,06:01
7,0.701840,1.687775,0.449385,06:01
8,0.675056,1.526904,0.506405,06:01
9,0.641715,1.452407,0.528762,06:02


In [34]:
df_test_valid.head()

,song_id,filename,filepath
0,7072,007072.ogg,test/007072.ogg
1,10207,010207.ogg,test/010207.ogg
2,20008,020008.ogg,test/020008.ogg
3,10924,010924.ogg,test/010924.ogg
4,21896,021896.ogg,test/021896.ogg


In [35]:
import gc
import torch
gc.collect()
torch.cuda.empty_cache()

### OOF

In [36]:
OOFOut = pd.DataFrame()
for fld in range(5):
    fold_num = fld
    kernel_type= 'densenet121d_MonoSpec_Hop448_Mels64_Noisy'
    OOF = np.load(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/oof_fold_{fold_num}_probs.npy',allow_pickle=True)
    ID = np.load(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/oof_fold_{fold_num}_fnames.npy',allow_pickle=True)
    OOF = ([item for sublist in OOF for item in sublist])
    OOF = pd.DataFrame(OOF)
    OOF['ID'] = ID
    OOFOut = pd.concat([OOF,OOFOut],0).reset_index(drop=True)


In [37]:
OOFOut = pd.merge(OOFOut,df_train[['song_id','genre_id']],left_on='ID',right_on='song_id',how='left')

In [38]:
OOF_preds=OOFOut.iloc[:,:19].idxmax(1)


In [39]:
from sklearn.metrics import f1_score

f1_score(OOFOut['genre_id'],OOF_preds,average="micro")

0.573086196503918

In [40]:
pd.crosstab(OOF_preds,OOFOut['genre_id'])

genre_id,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
row_0,,,,,,,,,,,,,,,,,,,
0,2019,137,46,177,263,21,374,107,146,99,401,4,0,34,0,36,0,2,5
1,112,1643,757,151,29,136,17,123,201,36,26,7,0,31,37,13,6,20,0
2,26,718,1578,42,11,15,4,17,63,16,0,0,1,2,11,3,2,0,0
3,201,151,74,1021,24,81,54,150,93,18,88,20,5,38,1,2,30,4,1
4,169,21,15,35,1345,5,12,5,19,18,19,0,0,2,0,4,3,6,1
5,14,111,21,69,6,785,1,42,101,32,6,8,2,11,24,3,5,7,0
6,175,13,3,49,16,0,678,5,11,0,23,1,0,0,0,0,0,0,0
7,104,79,8,90,10,54,7,471,30,4,70,40,1,26,1,4,0,2,1
8,66,109,38,38,23,61,8,16,212,24,5,1,0,4,11,6,0,4,0


### Mode based sub

In [41]:
kernel_type = 'densenet121d_MonoSpec_Hop448_Mels64_Noisy'

In [42]:
out = submission.copy().drop('genre_id',1)
for fld in range(5):
    fname = pd.read_csv(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/fold_{fld}.csv')
    out = pd.merge(out,fname[['song_id','target']],on='song_id',how='left')
    
submission['genre_id']=out.drop('song_id',1).mode(1)[0]

In [43]:
submission.loc[submission['song_id']==22612,'genre_id'] = 1
submission.loc[submission['song_id']==24013,'genre_id'] = 0
submission['genre_id'] = submission['genre_id'].astype(int)
submission.to_csv(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/5Fold_mode.csv',index=False)

### Mode based sub: check OOF Score

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from sklearn.metrics import f1_score

out = pd.DataFrame()
for kernels in [
                'eb1_MonoSpec_Hop448_Mels160',
                'resnest50d_MonoSpec_Hop448_Mels160_FTune',
                'resnest50d_MonoSpec_Hop448_Mels160',
#                 'resnet50d_MIXUP_finetune_MonoSpec_Hop512_Mels128',
                'tf_efficientnet_b1_MonoSpec_Hop512_Mels128',
#                 'densenet121d_MonoSpec_Hop448_Mels160_FTUNE',
#                 'resnest26d_MonoSpec_Hop448_Mels160_FTUNE',
                'ecaresnet50t_MonoSpec_Hop448_Mels160_FTUNE',
#                 'seresnet50t_MonoSpec_Hop512_Mels128',
#                 'seresnet50t_MonoSpec_Hop448_Mels160',
                'seresnet50t_MonoSpec_Hop448_Mels160_FTUNE',
                'ecaresnet50t_MonoSpec_Hop448_Mels160',
                'resnest50d_MonoSpec_Hop448_Mels160_NoisyTrain'
               ]:
    OOFOut = pd.DataFrame()
    for fld in range(5):
        fold_num = fld
        kernel_type= kernels
        OOF = np.load(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/oof_fold_{fold_num}_probs.npy',allow_pickle=True)
        ID = np.load(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/oof_fold_{fold_num}_fnames.npy',allow_pickle=True)
        OOF = ([item for sublist in OOF for item in sublist])
        OOF = pd.DataFrame(OOF)
        OOF['ID'] = ID
        OOFOut = pd.concat([OOF,OOFOut],0).reset_index(drop=True)

    OOFOut = pd.merge(OOFOut,df_train[['song_id','genre_id']],left_on='ID',right_on='song_id',how='left')
    OOF_preds=OOFOut.iloc[:,:19].idxmax(1)
    out = pd.concat([out,OOF_preds],1)
    out['genre_id'] = OOFOut['genre_id']
    
out['preds'] = out.drop('genre_id',1).mode(1)[0].astype(int)
f1_score(out['genre_id'],out['preds'],average="micro")

### Mode based sub

In [ ]:
kernel_type = 'resnest50d_MonoSpec_Hop448_Mels160_NoisyTrain'

In [ ]:
out = submission.copy().drop('genre_id',1)
for fld in range(5):
    for kernels in ['eb1_MonoSpec_Hop448_Mels160',
                'resnest50d_MonoSpec_Hop448_Mels160_FTune',
                'resnest50d_MonoSpec_Hop448_Mels160',
                'tf_efficientnet_b1_MonoSpec_Hop512_Mels128',
                'ecaresnet50t_MonoSpec_Hop448_Mels160_FTUNE',
                'seresnet50t_MonoSpec_Hop448_Mels160_FTUNE',
                'ecaresnet50t_MonoSpec_Hop448_Mels160',
                'resnest50d_MonoSpec_Hop448_Mels160_NoisyTrain'
               ]:
        fname = pd.read_csv(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernels}/fold_{fld}.csv')
        out = pd.merge(out,fname[['song_id','target']],on='song_id',how='left')
    
submission['genre_id']=out.drop('song_id',1).mode(1)[0]

In [ ]:
submission.loc[submission['song_id']==22612,'genre_id'] = 1
submission.loc[submission['song_id']==24013,'genre_id'] = 0
submission['genre_id'] = submission['genre_id'].astype(int)
submission.to_csv(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/5Fold_mode_ens_8bestmodels_CV_5862.csv',index=False)

In [ ]:
kernel_type

In [ ]:
from sklearn.metrics import f1_score

out = pd.DataFrame()
for kernels in [
                'eb1_MonoSpec_Hop448_Mels160',
                'resnest50d_MonoSpec_Hop448_Mels160_FTune',
#                 'resnest50d_MonoSpec_Hop448_Mels160',
#                 'resnet50d_MIXUP_finetune_MonoSpec_Hop512_Mels128',
                'tf_efficientnet_b1_MonoSpec_Hop512_Mels128',
#                 'densenet121d_MonoSpec_Hop448_Mels160_FTUNE',
#                 'resnest26d_MonoSpec_Hop448_Mels160_FTUNE',
                'ecaresnet50t_MonoSpec_Hop448_Mels160_FTUNE',
#                 'seresnet50t_MonoSpec_Hop448_Mels160_FTUNE',
#                 'ecaresnet50t_MonoSpec_Hop448_Mels160',
                'resnest50d_MonoSpec_Hop448_Mels160_NoisyTrain'
               ]:
    OOFOut = pd.DataFrame()
    for fld in range(5):
        fold_num = fld
        kernel_type= kernels
        OOF = np.load(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/oof_fold_{fold_num}_probs.npy',allow_pickle=True)
        ID = np.load(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/oof_fold_{fold_num}_fnames.npy',allow_pickle=True)
        OOF = ([item for sublist in OOF for item in sublist])
        OOF = pd.DataFrame(OOF)
        OOF['ID'] = ID
        OOFOut = pd.concat([OOF,OOFOut],0).reset_index(drop=True)

    OOFOut = pd.merge(OOFOut,df_train[['song_id','genre_id']],left_on='ID',right_on='song_id',how='left')
    OOF_preds=OOFOut.iloc[:,:19].idxmax(1)
    out = pd.concat([out,OOF_preds],1)
    out['genre_id'] = OOFOut['genre_id']
    
out['preds'] = out.drop('genre_id',1).mode(1)[0].astype(int)
f1_score(out['genre_id'],out['preds'],average="micro")

In [ ]:
kernel_type = 'resnest50d_MonoSpec_Hop448_Mels160_NoisyTrain'

In [ ]:
out = submission.copy().drop('genre_id',1)
for fld in range(5):
    for kernels in [
                'eb1_MonoSpec_Hop448_Mels160',
                'resnest50d_MonoSpec_Hop448_Mels160_FTune',
                'tf_efficientnet_b1_MonoSpec_Hop512_Mels128',
                'ecaresnet50t_MonoSpec_Hop448_Mels160_FTUNE',
                'resnest50d_MonoSpec_Hop448_Mels160_NoisyTrain'
               ]:
        fname = pd.read_csv(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernels}/fold_{fld}.csv')
        out = pd.merge(out,fname[['song_id','target']],on='song_id',how='left')
    
submission['genre_id']=out.drop('song_id',1).mode(1)[0]

In [ ]:
submission.loc[submission['song_id']==22612,'genre_id'] = 1
submission.loc[submission['song_id']==24013,'genre_id'] = 0
submission['genre_id'] = submission['genre_id'].astype(int)
submission.to_csv(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/5Fold_mode_ens_5bestmodels_CV_5877.csv',index=False)

### Avg probs

In [ ]:
# np.concatenate(np.load(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernels}/test_fold_0_probs.npy',allow_pickle=True)).shape,
df_test_valid.head()



In [ ]:
out = submission.copy().drop('genre_id',1)
f = np.zeros((df_test_valid.shape[0],19))
for fld in range(5):
    for kernels in [
                'eb1_MonoSpec_Hop448_Mels160',
                'resnest50d_MonoSpec_Hop448_Mels160_FTune',
                'tf_efficientnet_b1_MonoSpec_Hop512_Mels128',
                'ecaresnet50t_MonoSpec_Hop448_Mels160_FTUNE',
                'resnest50d_MonoSpec_Hop448_Mels160_NoisyTrain'
               ]:
        f += (np.concatenate(np.load(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernels}/test_fold_{fld}_probs.npy',allow_pickle=True)))/25
newDF = pd.DataFrame()
newDF['song_id'] =  df_test_valid['song_id']
newDF['genre_id'] = np.argmax(f,1)

out = pd.merge(out,newDF,on='song_id',how='left')    
# submission['genre_id']=out.drop('song_id',1).mode(1)[0]

In [ ]:
out.head()

In [ ]:
out.loc[submission['song_id']==22612,'genre_id'] = 1
out.loc[submission['song_id']==24013,'genre_id'] = 0
out['genre_id'] = out['genre_id'].astype(int)
out.to_csv(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/5Fold_mode_probEns_5bestmodels_NoCVInfo.csv',index=False)

#### Blind sub

In [ ]:
out = submission.copy().drop('genre_id',1)
f = np.zeros((df_test_valid.shape[0],19))
for fld in range(5):
    for kernels in [
                'eb1_MonoSpec_Hop448_Mels160',
                'resnest50d_MonoSpec_Hop448_Mels160_FTune',
                'resnest50d_MonoSpec_Hop448_Mels160',
                'tf_efficientnet_b1_MonoSpec_Hop512_Mels128',
                'ecaresnet50t_MonoSpec_Hop448_Mels160_FTUNE',
                'seresnet50t_MonoSpec_Hop448_Mels160_FTUNE',
                'ecaresnet50t_MonoSpec_Hop448_Mels160',
                'resnest50d_MonoSpec_Hop448_Mels160_NoisyTrain'
               ]:
        f += (np.concatenate(np.load(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernels}/test_fold_{fld}_probs.npy',allow_pickle=True)))/25
newDF = pd.DataFrame()
newDF['song_id'] =  df_test_valid['song_id']
newDF['genre_id'] = np.argmax(f,1)

out = pd.merge(out,newDF,on='song_id',how='left')    
# submission['genre_id']=out.drop('song_id',1).mode(1)[0]

In [ ]:
out.head()

In [ ]:
out.loc[submission['song_id']==22612,'genre_id'] = 1
out.loc[submission['song_id']==24013,'genre_id'] = 0
out['genre_id'] = out['genre_id'].astype(int)
out.to_csv(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/BlindSub_NoCVInfo.csv',index=False)

### End ###